In [1]:
# coding=utf-8
 
import numpy as np
import scipy as sp
import functools
 
nodes = []  
nodesnum = 0
nodes_index = {}  
damp = 0.8  
trans_matrix = np.matrix(0)  
sim_matrix = np.matrix(0)  
 
 
def initParam(graphFile):
   
    global nodes
    global nodes_index
    global trans_matrix
    global sim_matrix
    global damp
    global nodesnum
 
    link_in = {}
    for line in open(graphFile, "r", 1024):
        arr = line.strip("\n").split()
        node = arr[0]
        nodeid = -1
        if node in nodes_index:
            nodeid = nodes_index[node]
        else:
            nodeid = len(nodes)
            nodes_index[node] = nodeid
            nodes.append(node)
        for ele in arr[1:]:
            outneighbor = ele
            outneighborid = -1
            if outneighbor in nodes_index:
                outneighborid = nodes_index[outneighbor]
            else:
                outneighborid = len(nodes)
                nodes_index[outneighbor] = outneighborid
                nodes.append(outneighbor)
            inneighbors = []
            if outneighborid in link_in:
                inneighbors = link_in[outneighborid]
            inneighbors.append(nodeid)
            link_in[outneighborid] = inneighbors
 
    nodesnum = len(nodes)
    trans_matrix = np.zeros((nodesnum, nodesnum))
    for node, inneighbors in link_in.items():
        num = len(inneighbors)
        prob = 1.0 / num
        for neighbor in inneighbors:
            trans_matrix[neighbor, node] = prob
 
    sim_matrix = np.identity(nodesnum) * (1 - damp)
 
 
def iterate():

    global trans_matrix
    global sim_matrix
    global damp
    global nodesnum
 
    sim_matrix = damp * np.dot(np.dot(trans_matrix.transpose(),
                                      sim_matrix), trans_matrix) + (1 - damp) * np.identity(nodesnum)
 
 
def printResult(sim_node_file):
    
    global sim_matrix
    global link_out
    global link_in
    global nodes
    global nodesnum
    
    f_out_user = open(sim_node_file, "w")
    for i in range(nodesnum):
        f_out_user.write(nodes[i] + "\t")
        neighbour = []
        for j in range(nodesnum):
            if i != j:
                sim = sim_matrix[i, j]
                if sim == None:
                    sim = 0
                if sim > 0:
                    neighbour.append((j, sim))
#         neighbour = sorted(neighbour, key=functools.cmp_to_key(lambda x, y: cmp(x[1], y[1])), reverse=True)
        neighbour = sorted(neighbour, key=Key, reverse=True)
        for (u, sim) in neighbour:
            f_out_user.write(nodes[u] + ":" + str(sim) + "\t")
        f_out_user.write("\n")
    f_out_user.close()
 
 
def simrank(graphFile, maxIteration):
    global nodes_index
    global trans_matrix
    global sim_matrix
 
    initParam(graphFile)
    print( "nodes:")
    print(nodes_index)
    print("trans ratio:")
    print(trans_matrix)
    for i in range(maxIteration):
        print("iteration %d:" % (i + 1))
        iterate()
        print(sim_matrix)
 
class Key(str): 
      def __lt__(x, y): 
          return x[1] <y[1] 
    
if __name__ == '__main__':
    graphFile = "/Users/ninashao/Downloads/data_sample/MS_sample/relationship_total.txt"
    sim_node_file = "/Users/ninashao/Downloads/data_sample/MS_sample/nodesim_naive"
    maxIteration = 10
    simrank(graphFile, maxIteration)
    printResult(sim_node_file)

nodes:
{'16724': 1113, '25608': 2217, '37697': 3775, '37842': 3795, '18393': 1304, '27363': 2471, '1295': 117, '41671': 4280, '28141': 2572, '22536': 1820, '42039': 4331, '25610': 2218, '33505': 3266, '34483': 3370, '1134': 163, '35352': 3484, '31476': 2998, '39086': 3950, '21823': 1727, '32647': 3159, '18056': 1274, '27795': 2524, '18559': 1324, '26534': 2350, '30134': 2851, '15518': 958, '35669': 3520, '26068': 2283, '41240': 4223, '1102': 300, '20037': 1508, '34204': 3345, '40792': 4166, '10010': 0, '37156': 3702, '34232': 3348, '15294': 928, '22080': 1758, '29726': 2786, '38078': 3817, '39701': 4015, '35515': 3503, '42474': 4388, '32067': 3083, '42501': 4392, '12583': 559, '18828': 1351, '41601': 4271, '12933': 607, '33286': 3238, '28284': 2594, '37378': 3738, '11142': 323, '31757': 3036, '29952': 2825, '27509': 2488, '25874': 2253, '33307': 3241, '20147': 1524, '36812': 3659, '1016': 190, '25179': 2155, '37800': 3791, '27121': 2433, '29462': 2749, '29706': 2782, '16414': 1078, '13

[[  2.14545455e-01   0.00000000e+00   0.00000000e+00 ...,   3.63636364e-03
    9.69696970e-03   2.42424242e-03]
 [  0.00000000e+00   2.00567376e-01   2.57248513e-05 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   2.57248513e-05   2.00403023e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  3.63636364e-03   0.00000000e+00   0.00000000e+00 ...,   2.20000000e-01
    3.33333333e-03   0.00000000e+00]
 [  9.69696970e-03   0.00000000e+00   0.00000000e+00 ...,   3.33333333e-03
    2.26666667e-01   4.44444444e-03]
 [  2.42424242e-03   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    4.44444444e-03   2.26666667e-01]]
iteration 2:
[[  2.14704797e-01   0.00000000e+00   0.00000000e+00 ...,   3.66648189e-03
    9.73384140e-03   2.44718568e-03]
 [  0.00000000e+00   2.04711432e-01   4.08524232e-03 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   4.08524232e-03   2.04444653e-01 ...,   0.00000000e+00
    0.00